In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
# sl=eval(input('请输入边长：'))
# fp=(eval(input('请输入旗点纵坐标（从0开始）：')), eval(input('请输入旗点横坐标（从0开始）：')))

# mat=[]
# for i in range(sl):
#     mat.append(input('请输入第{}行（空格分隔）：'.format(i+1)).split())

# mat=pd.DataFrame(mat)

# mat

In [ ]:
fp=(eval(input('请输入旗点纵坐标（从0开始）：')), eval(input('请输入旗点横坐标（从0开始）：')))

mat=pd.read_excel(r'目录/地图.xlsx', header=None)
sl=np.shape(mat)[0]

mat

In [ ]:
G=nx.Graph()

for i in range(sl):
    for j in range(sl):
        G.add_node((i,j), color=mat.iloc[i,j])

for i in range(sl):
    for j in range(sl):
        if j+1<=sl-1:
            G.add_edge((i,j), (i,j+1))
        if i+1<=sl-1:
            G.add_edge((i,j), (i+1,j))

# print(G._node)
# print(pd.DataFrame(G.edges))

In [ ]:
def func(G, fp, seq):
    G1=G.copy()
    steps=np.inf
    counter=0
    
    # 依次执行操作序列中的操作
    for i in range(len(seq)+1):
        # 遍历所有节点
        for node in list(G1._node):
            # 防止把旗点所在色块对应的节点弄没了
            if node!=fp:
                # 遍历当前节点的相邻节点
                for neighbor in G1[node]:
                    # 若当前节点的相邻节点中有与其同色的节点
                    if (G1._node[node])['color']==(G1._node[neighbor])['color']:
                        # 将该相邻节点设为目标节点
                        target_neighbor=neighbor
                        # 将其他相邻节点的名称存入列表
                        other_neighbors=list(set(G1[node])-{target_neighbor})
                        
                        # 建立其他相邻节点与目标节点间的边
                        for curr_neighbor in other_neighbors:
                            G1.add_edge(curr_neighbor, target_neighbor)
                            
                        # 删除当前节点
                        G1.remove_node(node)
                        break
        
        # 若节点数减小到了1，说明已通关
        if len(G1._node)==1:
            steps=counter
            break
        
        # 将旗点所在色块对应的节点变色
        (G1._node[fp])['color']=seq[i] if i<=len(seq)-1 else list(seq)[-1]
        
        counter+=1
        
    return steps

In [ ]:
def seq_generator(G, fp, n, l):
    pre_color=(G._node[fp])['color']
    seq_mat=[]
    for i in range(n):
        seq_ls=[]
        for j in range(l):
            colors={'r','o','y','g','b','p'}-set(pre_color)
            curr_color=np.random.choice(list(colors))
            seq_ls.append(curr_color)
            pre_color=curr_color
        
        seq_mat.append(seq_ls)
        
    return pd.DataFrame(seq_mat)

In [ ]:
def GA(G, fp, seq_num, seq_len, death_num, prob_mut, target_steps, max_iter, return_log=False):
    seq_opt=[]
    min_steps=np.inf
    seq_df=seq_generator(G, fp, seq_num, seq_len)
    steps_of_curr_seqs=pd.DataFrame(np.empty((seq_num, 1), dtype=int), columns=['steps'])
    log=[]
    
    for i in range(max_iter+1):
        for j in range(seq_num):
            steps_of_curr_seqs.iloc[j,0]=func(G, fp, seq_df.iloc[j,:])
            
        df_sorted=pd.concat([seq_df, steps_of_curr_seqs], axis=1).sort_values(by='steps')
        log.append([i, df_sorted['steps'].iloc[0]])
        
        if df_sorted['steps'].iloc[0]<min_steps:
            min_steps=df_sorted['steps'].iloc[0]
            seq_opt=list(df_sorted.iloc[0,:-1])
        
        if min_steps<=target_steps:
            break
        
        seq_alive_df=df_sorted.iloc[0:seq_num-death_num, :-1]
        
        seq_new_ls=[]
        for j in range(death_num):
            parents_index=np.random.choice(range(seq_num-death_num), 2)
            
            cut_positions=[]
            for k in range(seq_len-1):
                if seq_alive_df.iloc[parents_index[0], k]!=seq_alive_df.iloc[parents_index[1], k+1]:
                    cut_positions.append(k)
                    
            cut_position=np.random.choice(cut_positions) if cut_positions!=[] else np.random.choice(range(seq_len-1))
            
            newseq_head=list(seq_alive_df.iloc[parents_index[0], :cut_position+1])
            newseq_tail=list(seq_alive_df.iloc[parents_index[1], cut_position+1:])
            seq_new_ls.append(newseq_head+newseq_tail)
        
        seq_new_df=pd.DataFrame(seq_new_ls)
        seq_df_new=pd.concat([seq_alive_df, seq_new_df], axis=0)
        
        mut_num=round(seq_num*prob_mut)
        mut_seq_index=np.random.choice(range(seq_num), mut_num)
        
        for j in range(mut_num):
            mut_position=np.random.randint(0, seq_len)
            seq_df_new.iloc[mut_seq_index[j], mut_position]=np.random.choice(['r','o','y','g','b','p'])
            
        seq_df=seq_df_new
        seq_df.index=range(seq_num)
        
    if return_log==True:
        return min_steps, seq_opt, pd.DataFrame(log, columns=['iterations','curr_min_steps'])
    else:
        return min_steps, seq_opt

In [ ]:
# G, fp, seq_num, seq_len, death_num, prob_mut, target_steps, max_iter, return_log
min_steps, seq_opt, log=GA(G, fp, 1600, 16, 800, 0.01, 12, 100, True)

print(min_steps)
print(seq_opt)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

plt.plot(log['iterations'], log['curr_min_steps'])
plt.xlabel('iterations')
plt.ylabel('curr_min_steps')